In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from selenium.webdriver.common.by import By
import json
import pygsheets
from google.oauth2 import service_account
import platform
import datetime
from dateutil.relativedelta import relativedelta
import sys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

class MyChromeDriver(webdriver.Chrome):

    def __init__(self, headless=False, maximize=True):

        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('--headless')

        webdriver.Chrome.__init__(self, service=ChromeService(ChromeDriverManager().install()), options=options)

        if maximize:
            self.maximize_window()

os.getcwd()

'c:\\Users\\SANDBOX\\Desktop\\Sandbox\\crawling'

In [2]:
# 게이밍 시트 가져오기 : 시트ID로
gc = pygsheets.authorize(service_account_file='creds.json')

sheet_id = '1w_JkKqp08e_fV_fPJHRKqwS1FXg7jfyy5ycNXDKdwNw'
wks = gc.open_by_key(sheet_id)
sh = wks.worksheet_by_title('크롤링 리스트(23.4Q)')
gaming = sh.get_as_df(has_header=True , index_column=None , start='A1', end='H100000' , numerize=True , empty_value=np.nan)

# 크롤링 기준
gaming_0_1 = gaming[(gaming['게임 광고'] == 0) | (gaming['게임 광고'] == 1)]
gaming_0_1
# gaming_0_1.to_csv('게이밍 크롤링 리스트')

,채널명,채널ID,구독자,유튜브 링크,게임 광고,중복 체크,분기
148,김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,"1,740,000",https://www.youtube.com/channel/UCNzcxCN_Hh_lu...,1,중복X,NaN
149,홍방장,UCHxrU6bjWrnBfmReUAeztcw,"272,000",https://www.youtube.com/channel/UCHxrU6bjWrnBf...,1,중복X,NaN
150,침착맨,UCUj6rrhMTR9pipbAWBAMvUQ,"2,260,000",https://www.youtube.com/channel/UCUj6rrhMTR9pi...,1,중복X,NaN
151,오킹,UCom6YhUY62jM52nIMjf5_dw,"2,080,000",https://www.youtube.com/channel/UCom6YhUY62jM5...,1,중복X,NaN
152,김달걀,UCfNUrgI9-Q5ySUJo2VUoX1Q,"81,500",https://www.youtube.com/channel/UCfNUrgI9-Q5yS...,1,중복X,NaN
...,...,...,...,...,...,...,...
591,사나이 김기훈,UC9nFMAQcn0bqIrEV9vUdU1g,"1,160,000",https://www.youtube.com/channel/UC9nFMAQcn0bqI...,0,중복X,NaN
592,라로시 Larothy,UCJXbMRXtX6vHDkGoeerk8Yw,"4,480",https://www.youtube.com/channel/UCJXbMRXtX6vHD...,0,중복X,NaN
593,싸패TV,UCfycsfXKilVd-EBL9VC8RGA,"97,000",https://www.youtube.com/channel/UCfycsfXKilVd-...,0,중복X,NaN
594,블러비BLOVEY,UCXz_coEujrsE9WMJJbg6lTg,"88,900",https://www.youtube.com/channel/UCXz_coEujrsE9...,0,중복X,NaN


In [3]:
len(gaming_0_1['채널ID'].unique())

446

In [4]:
# 각 채널의 3개월 전 영상 크롤링
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
if driver is None:
        my_driver = MyChromeDriver()
        time.sleep(10)
else:
    my_driver = driver
    
channel_id_list = gaming_0_1['채널ID'].unique().tolist()
video_date_list = []
video_view_list = []
video_url_list = []
youtube_video_url_dic = {}

for i in tqdm(range(len(channel_id_list))):
    url = 'https://www.youtube.com/channel/' + channel_id_list[i] + '/videos'
    driver.get(url)
    time.sleep(3)
    print(i, url)

    pageString = driver.page_source
    soup = BeautifulSoup(pageString, features='html.parser')

    date_count = 0
    date_list = ['분 전', '시간 전', '1일 전', '2일 전', '3일 전', '4일 전', '5일 전', '6일 전', '7일 전', 
                 '8일 전', '9일 전', '10일 전', '11일 전', '12일 전', '13일 전', '2주 전', '3주 전', '4주 전', 
                 '1개월 전', '2개월 전', '3개월 전']

    # 날짜, 조회수, URL
    date_soup = soup.find_all('span', attrs={'class' : 'inline-metadata-item style-scope ytd-video-meta-block'})[1::2]
    view_soup = soup.find_all('span', attrs={'class' : 'inline-metadata-item style-scope ytd-video-meta-block'})[0::2]
    url_soup = soup.find_all('a', attrs={'class': 'yt-simple-endpoint focus-on-expand style-scope ytd-rich-grid-media'})

    # 날짜 기준에 맞는 영상 개수 카운트
    for j in range(len(date_soup)):
        if date_soup[j].text in date_list:
            date_count += 1
        if date_list[0] in date_soup[j].text:
            date_count += 1

    # 날짜 기준에 맞도록 저장
    date_soup = date_soup[:date_count]
    view_soup = view_soup[:date_count]
    url_soup = url_soup[:date_count]

    # 영상 정보 및 URL 저장
    for k in range(len(date_soup)):
        video_date = date_soup[k].text
        video_date_list.append(video_date)

        video_view = view_soup[k].text.replace('조회수 ', '').replace('회', '')
        video_view_list.append(video_view)

        video_url = 'https://www.youtube.com' + url_soup[k]['href']
        video_url_list.append(video_url)
        print(video_date, video_view, video_url)

    youtube_video_url_dic[channel_id_list[i]] = video_date_list
    youtube_video_url_dic[channel_id_list[i]] = video_view_list
    youtube_video_url_dic[channel_id_list[i]] = video_url_list

driver.quit()

  0%|          | 0/446 [00:00<?, ?it/s]

0 https://www.youtube.com/channel/UCNzcxCN_Hh_lu5RCSFXKgGQ/videos


  0%|          | 1/446 [00:06<46:40,  6.29s/it]

1일 전 13만 https://www.youtube.com/watch?v=Ohubfyb9jds
3일 전 13만 https://www.youtube.com/watch?v=_HXjvJre59c
4일 전 5.8만 https://www.youtube.com/watch?v=sFzDgCaLTrk
6일 전 11만 https://www.youtube.com/watch?v=WlepEAy0FyU
8일 전 13만 https://www.youtube.com/watch?v=TRQVdAWMiro
10일 전 15만 https://www.youtube.com/watch?v=INI1PdsXQus
11일 전 31만 https://www.youtube.com/watch?v=HxT9AdEHrHw
2주 전 8.3만 https://www.youtube.com/watch?v=EPOb0GqJ39E
2주 전 16만 https://www.youtube.com/watch?v=mgN13oV9c0A
2주 전 25만 https://www.youtube.com/watch?v=Y6wnJTwADLI
2주 전 18만 https://www.youtube.com/watch?v=dW-aBQK_V3Y
2주 전 13만 https://www.youtube.com/watch?v=Mbsqzi5Qu7A
3주 전 17만 https://www.youtube.com/watch?v=4CRp8WRT1J0
3주 전 22만 https://www.youtube.com/watch?v=vjP5Fo8m3mA
3주 전 16만 https://www.youtube.com/watch?v=IUujyzK_s7g
4주 전 12만 https://www.youtube.com/watch?v=ejrnTAwI0OE
4주 전 16만 https://www.youtube.com/watch?v=Ti7ku8u6ZuM
1개월 전 27만 https://www.youtube.com/watch?v=GcK02la15j4
1개월 전 7.5만 https://www.youtube.com/watch?

  0%|          | 2/446 [00:11<41:25,  5.60s/it]

...

3개월 전 1.2만 https://www.youtube.com/watch?v=otLfBZQMZnc
444 https://www.youtube.com/channel/UCXz_coEujrsE9WMJJbg6lTg/videos


100%|█████████▉| 445/446 [38:57<00:04,  4.81s/it]

2일 전 798 https://www.youtube.com/watch?v=XX1gcy-wNrc
2주 전 2.2천 https://www.youtube.com/watch?v=3YPgDVWFEic
1개월 전 2.2천 https://www.youtube.com/watch?v=Mple6pE71mg
1개월 전 3.4천 https://www.youtube.com/watch?v=zbtvofQO6Fc
1개월 전 2.2천 https://www.youtube.com/watch?v=D1o5xXrAye4
1개월 전 4.3만 https://www.youtube.com/watch?v=bRCeCQufbTc
1개월 전 3.1만 https://www.youtube.com/watch?v=LoV5-jP9xoI
2개월 전 3.6천 https://www.youtube.com/watch?v=hN4FFslOIPU
2개월 전 2.7천 https://www.youtube.com/watch?v=axWfPZOpzMM
2개월 전 3.6천 https://www.youtube.com/watch?v=OyPexliD-P0
3개월 전 1.4만 https://www.youtube.com/watch?v=uc1Nf1bCXgw
3개월 전 2.5천 https://www.youtube.com/watch?v=0rUPpS9zAe0
3개월 전 3.7천 https://www.youtube.com/watch?v=j43aZrtnBTU
3개월 전 4.5천 https://www.youtube.com/watch?v=AF0PJMzZxFc
445 https://www.youtube.com/channel/UCPZIPuQPrfrUG9Xe_okEmQA/videos


100%|██████████| 446/446 [39:02<00:00,  5.25s/it]

1일 전 6.2만 https://www.youtube.com/watch?v=YAjJ5s5cQM0
1일 전 6.8만 https://www.youtube.com/watch?v=pdaB7d8T91o
5일 전 5.5만 https://www.youtube.com/watch?v=gp6guHnk75c
7일 전 3.9만 https://www.youtube.com/watch?v=YdlV0kHxi18
13일 전 3.9만 https://www.youtube.com/watch?v=dkQPnZDcOgk
3주 전 7.5만 https://www.youtube.com/watch?v=Ur7rv0_Zk5M
1개월 전 285만 https://www.youtube.com/watch?v=Rb92-wFteQE
1개월 전 30만 https://www.youtube.com/watch?v=YlXC3Putc3k
1개월 전 475만 https://www.youtube.com/watch?v=c_yCRwh97M8
1개월 전 21만 https://www.youtube.com/watch?v=NHS99uKCV7E
1개월 전 19만 https://www.youtube.com/watch?v=3gAi4kYwbdU
2개월 전 7.1만 https://www.youtube.com/watch?v=9wCfL3EJbhI
2개월 전 38만 https://www.youtube.com/watch?v=9j9_8EONCk4
2개월 전 4.8만 https://www.youtube.com/watch?v=bmdXTqwQIMQ
2개월 전 219만 https://www.youtube.com/watch?v=jDQ4-C4AHmA
2개월 전 10만 https://www.youtube.com/watch?v=i5pVLNyLGHg
2개월 전 21만 https://www.youtube.com/watch?v=5ZYmuugIres
3개월 전 7.1만 https://www.youtube.com/watch?v=XenmgneevLE
3개월 전 9.3만 https://ww

In [7]:
len(video_url_list)

7628

In [8]:
# 데이터프레임 생성
data = {'날짜': video_date_list, 
        '조회수': video_view_list, 
        '비디오URL': video_url_list}
gaming_df = pd.DataFrame(data)

# 데이터프레임 출력
gaming_df

,날짜,조회수,비디오URL
0,1일 전,13만,https://www.youtube.com/watch?v=Ohubfyb9jds
1,3일 전,13만,https://www.youtube.com/watch?v=_HXjvJre59c
2,4일 전,5.8만,https://www.youtube.com/watch?v=sFzDgCaLTrk
3,6일 전,11만,https://www.youtube.com/watch?v=WlepEAy0FyU
4,8일 전,13만,https://www.youtube.com/watch?v=TRQVdAWMiro
...,...,...,...
7623,2개월 전,10만,https://www.youtube.com/watch?v=i5pVLNyLGHg
7624,2개월 전,21만,https://www.youtube.com/watch?v=5ZYmuugIres
7625,3개월 전,7.1만,https://www.youtube.com/watch?v=XenmgneevLE
7626,3개월 전,9.3만,https://www.youtube.com/watch?v=GNVGHhWTm44


In [9]:
# youtube_video_url_dic 사용하는 경우
gaming_video_df = pd.DataFrame()

for i in range(len(youtube_video_url_dic.keys())):
    new_df = pd.DataFrame({'channel id':[list(youtube_video_url_dic.keys())[i]] * len(youtube_video_url_dic[list(youtube_video_url_dic.keys())[i]]), 
                           'url': youtube_video_url_dic[list(youtube_video_url_dic.keys())[i]]})
    gaming_video_df = pd.concat([gaming_video_df, new_df])
    
gaming_video_df = gaming_video_df.reset_index(drop=True)
gaming_video_df

,channel id,url
0,UCNzcxCN_Hh_lu5RCSFXKgGQ,https://www.youtube.com/watch?v=Ohubfyb9jds
1,UCNzcxCN_Hh_lu5RCSFXKgGQ,https://www.youtube.com/watch?v=_HXjvJre59c
2,UCNzcxCN_Hh_lu5RCSFXKgGQ,https://www.youtube.com/watch?v=sFzDgCaLTrk
3,UCNzcxCN_Hh_lu5RCSFXKgGQ,https://www.youtube.com/watch?v=WlepEAy0FyU
4,UCNzcxCN_Hh_lu5RCSFXKgGQ,https://www.youtube.com/watch?v=TRQVdAWMiro
...,...,...
3402083,UCPZIPuQPrfrUG9Xe_okEmQA,https://www.youtube.com/watch?v=i5pVLNyLGHg
3402084,UCPZIPuQPrfrUG9Xe_okEmQA,https://www.youtube.com/watch?v=5ZYmuugIres
3402085,UCPZIPuQPrfrUG9Xe_okEmQA,https://www.youtube.com/watch?v=XenmgneevLE
3402086,UCPZIPuQPrfrUG9Xe_okEmQA,https://www.youtube.com/watch?v=GNVGHhWTm44


In [15]:
video_url_list = gaming_df['비디오URL'].tolist()

In [16]:
# 영상 유료광고 배너 크롤링

driver = webdriver.Chrome()

video_url_list_2 = []
channel_list = []
channel_id_list = []
sub_list = []
ad_banner_list = []

for i in tqdm(range(len(video_url_list))):
    driver.get(video_url_list[i])
    time.sleep(3)
    
    try:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 유료 광고 영상 배너 확인
        ad_banner = soup.find_all('div', class_='ytp-paid-content-overlay-text')
        
        if ad_banner:
            # 채널명
            try:
                channel = soup.find_all('yt-formatted-string', {'id': 'text'})[0].text
            except:
                channel = '채널명 크롤링 오류'
            
            # 채널ID
            try:
                channel_id = str(soup).split(',"channelId":"')[1].split('","')[0]
            except:
                channel_id = '채널ID 크롤링 오류'
            
            # 구독자 수
            try:
                sub = soup.find('yt-formatted-string', {'id': 'owner-sub-count'}).text.split('구독자')[1].strip().split('명')[0].strip()
            except:
                sub = '구독자 수 크롤링 오류'
            
            # 모든 데이터가 성공적으로 크롤링되면 리스트에 추가
            video_url_list_2.append(video_url_list[i])
            channel_list.append(channel)
            channel_id_list.append(channel_id)
            sub_list.append(sub)
            ad_banner_list.append(ad_banner[0].text)

            print(i, video_url_list_2[-1], ad_banner_list[-1])
            
    except Exception as e:
        print("크롤링 오류:", e)
    
driver.quit()

  0%|          | 1/7628 [00:08<18:11:14,  8.58s/it]

0 https://www.youtube.com/watch?v=Ohubfyb9jds 


  0%|          | 2/7628 [00:16<16:45:11,  7.91s/it]

1 https://www.youtube.com/watch?v=_HXjvJre59c 


  0%|          | 3/7628 [00:21<14:08:09,  6.67s/it]

...

7625 https://www.youtube.com/watch?v=XenmgneevLE 


100%|█████████▉| 7627/7628 [11:10:12<00:05,  5.34s/it]

7626 https://www.youtube.com/watch?v=GNVGHhWTm44 


100%|██████████| 7628/7628 [11:10:18<00:00,  5.27s/it]

7627 https://www.youtube.com/watch?v=232s1lxPyDE 


In [17]:
print(len(video_url_list), len(video_url_list_2), len(channel_list), len(channel_id_list), len(sub_list), len(ad_banner_list))
# 만약 크롤링 중간에 멈추거나 오류나면 : 이미 크롤링한 데이터까지 저장 후 시트로 내보내서 광고 개수 구하기
# 크롤링 안 된 채널 gaming_df에서 비디오 url 매칭시켜서 따로 코드 돌리기

7628 7628 7628 7628 7628 7628


In [18]:
# 전체 데이터프레임 생성
data_2 = {'비디오URL': video_url_list_2,
          '채널명': channel_list,
          '채널ID': channel_id_list,
          '구독자 수': sub_list,
          '유료광고 포함 여부': ad_banner_list}
gaming_df_2 = pd.DataFrame(data_2)

# 데이터프레임 출력
gaming_df_2

,비디오URL,채널명,채널ID,구독자 수,유료광고 포함 여부
0,https://www.youtube.com/watch?v=Ohubfyb9jds,악동 김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,174만,
1,https://www.youtube.com/watch?v=_HXjvJre59c,악동 김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,174만,
2,https://www.youtube.com/watch?v=sFzDgCaLTrk,악동 김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,174만,
3,https://www.youtube.com/watch?v=WlepEAy0FyU,악동 김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,174만,
4,https://www.youtube.com/watch?v=TRQVdAWMiro,악동 김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,174만,
...,...,...,...,...,...
7623,https://www.youtube.com/watch?v=i5pVLNyLGHg,PLAVE 플레이브,UCPZIPuQPrfrUG9Xe_okEmQA,47.1만,
7624,https://www.youtube.com/watch?v=5ZYmuugIres,PLAVE 플레이브,UCPZIPuQPrfrUG9Xe_okEmQA,47.1만,
7625,https://www.youtube.com/watch?v=XenmgneevLE,PLAVE 플레이브,UCPZIPuQPrfrUG9Xe_okEmQA,47.1만,
7626,https://www.youtube.com/watch?v=GNVGHhWTm44,PLAVE 플레이브,UCPZIPuQPrfrUG9Xe_okEmQA,47.1만,


In [19]:
final = gaming_df_2[(gaming_df_2['유료광고 포함 여부'] == '유료 광고 포함')]
final

,비디오URL,채널명,채널ID,구독자 수,유료광고 포함 여부
16,https://www.youtube.com/watch?v=Ti7ku8u6ZuM,악동 김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,174만,유료 광고 포함
39,https://www.youtube.com/watch?v=ppUYHk0Cm6g,홍방장,UCHxrU6bjWrnBfmReUAeztcw,27.2만,유료 광고 포함
40,https://www.youtube.com/watch?v=fWE34lzTyRI,홍방장,UCHxrU6bjWrnBfmReUAeztcw,27.2만,유료 광고 포함
55,https://www.youtube.com/watch?v=ehJF_iMoEDM,침착맨,UCUj6rrhMTR9pipbAWBAMvUQ,226만,유료 광고 포함
60,https://www.youtube.com/watch?v=bG_CVIf51GM,침착맨,UCUj6rrhMTR9pipbAWBAMvUQ,226만,유료 광고 포함
...,...,...,...,...,...
7546,https://www.youtube.com/watch?v=n1d_j6-aJLg,씨랙,UCk-QtahVyHaxCsteeKB2Xzg,12.4만,유료 광고 포함
7550,https://www.youtube.com/watch?v=Vm8RUn1dcWs,씨랙,UCk-QtahVyHaxCsteeKB2Xzg,12.4만,유료 광고 포함
7576,https://www.youtube.com/watch?v=zik_nBEshqY,사나이 김기훈,UC9nFMAQcn0bqIrEV9vUdU1g,116만,유료 광고 포함
7577,https://www.youtube.com/watch?v=FExYY26f978,사나이 김기훈,UC9nFMAQcn0bqIrEV9vUdU1g,116만,유료 광고 포함


In [22]:
# 결과 구글 시트에 저장
gc = pygsheets.authorize(service_account_file='creds.json')
sheet_id = '1w_JkKqp08e_fV_fPJHRKqwS1FXg7jfyy5ycNXDKdwNw'
sh = gc.open_by_key(sheet_id)

# 오류 발생 시, worksheet의 행 수 늘려서 재실행
wks = sh.worksheet_by_title('크롤링 0/1(23.4Q)') # wks = sh.add_worksheet()
wks.set_dataframe(final, 'A1', index=False)

# # 결과 구글 시트에 저장
# gc = pygsheets.authorize(service_account_file='creds.json')
# sheetname= '크롤링 시도'
# sh = gc.open(sheetname)

# # 오류 발생 시, worksheet의 행 수 늘려서 재실행
# wks = sh.worksheet_by_title('Gaming 0/1')
# wks.set_dataframe(filtered_df, 'A1', index=False)